File Created just to better understand pandas dataframe structure

In [21]:
import pandas as pd
import os
from fastapi import HTTPException
from datetime import datetime
from pydantic import BaseModel
from typing import List, Optional

class VehicleData(BaseModel):
    make: str
    model: str
    year: int
    mileage: Optional[float] = None
    engine_size: Optional[float] = None

# Path to the scoring variables CSV
SCORING_VARIABLES_FILE = "app/data/scoring_variables.csv"  # Use forward slashes (cross-platform)

# Load the scoring variables CSV into a DataFrame
try:
    scoring_variables_df = pd.read_csv(SCORING_VARIABLES_FILE)
    print(f"Scoring Variables Loaded:\n{scoring_variables_df}")
except Exception as e:
    print(f"Error loading scoring variables: {e}")
    scoring_variables_df = None

def calculate_score_from_data(v: VehicleData) -> float:
    """
    Calculate the score for a vehicle based on dynamic scoring variables loaded from CSV.
    """
    if scoring_variables_df is None:
        raise HTTPException(status_code=500, detail="Scoring variables not loaded.")

    make_model_key = f"{v.make}_{v.model}".replace(" ", "_")

    if make_model_key not in scoring_variables_df.columns:
        raise HTTPException(
            status_code=422,
            detail=f"Scoring rules not found for make/model: {make_model_key}"
        )

    variables = scoring_variables_df.set_index("variable")[make_model_key]
    print(f"\nScoring Variables for {make_model_key}:\n{variables}\n")

    total_score = 0.0

    # Age-based score
    age_weight = float(variables["age_weight"])
    age = datetime.utcnow().year - v.year
    print(f"\nAge Weight: {age}\n") # Debug print to check age weight
    total_score += max(0, age) * age_weight
    print(f"Age Contribution: {max(0, age)} * {age_weight} = {max(0, age) * age_weight}")

    # Mileage-based score
    if v.mileage is not None:
        mileage_bonus = float(variables["mileage_bonus"])
        mileage_contribution = (100_000 - v.mileage) * mileage_bonus
        total_score += mileage_contribution
        print(f"Mileage Contribution: (100000 - {v.mileage}) * {mileage_bonus} = {mileage_contribution}")

    # Engine size-based score
    if v.engine_size is not None:
        engine_size_bonus = float(variables["engine_size_bonus"])
        engine_size_contribution = v.engine_size * engine_size_bonus
        total_score += engine_size_contribution
        print(f"Engine Size Contribution: {v.engine_size} * {engine_size_bonus} = {engine_size_contribution}")

    return round(total_score, 2)

# Test the function with a sample vehicle
vehicle = VehicleData(make="Toyota", model="Camry", year=2020, mileage=10000, engine_size=2.5)
score = calculate_score_from_data(vehicle)

print(f"\nFinal Score: {score}")


Scoring Variables Loaded:
            variable  Toyota_Camry  Honda_Civic  Ford_F150
0         age_weight         1.700        1.600      2.000
1      mileage_bonus         0.015        0.012      0.008
2  engine_size_bonus         0.100        0.050      0.200

Scoring Variables for Toyota_Camry:
variable
age_weight           1.700
mileage_bonus        0.015
engine_size_bonus    0.100
Name: Toyota_Camry, dtype: float64


Age Weight: 5

Age Contribution: 5 * 1.7 = 8.5
Mileage Contribution: (100000 - 10000.0) * 0.015 = 1350.0
Engine Size Contribution: 2.5 * 0.1 = 0.25

Final Score: 1358.75


C:\Users\steph\AppData\Local\Temp\ipykernel_46852\3955190118.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  age = datetime.utcnow().year - v.year
